In [1]:
import psycopg2
import pandas as pd

In [2]:
def create_database():
    # connect to default database
    # conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    #create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS accounts")
    cur.execute("CREATE DATABASE accounts")
    
    # close connection to default database
    conn.close()
    
    # connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=accounts user=postgres password=root")
    cur = conn.cursor()
    
    return cur, conn

In [3]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [5]:
HousePrices = pd.read_csv("D:/Shaniel/Data Engineer Projects/Data_Eng_Project_1/submission.csv")

In [6]:
HousePrices.head()

,Id,SalePrice
0,1461,122186.0
1,1462,160592.0
2,1463,185391.0
3,1464,193142.0
4,1465,186939.0


In [7]:
HouseTrain = pd.read_csv("D:/Shaniel/Data Engineer Projects/Data_Eng_Project_1/train.csv")

In [8]:
HouseTrain.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,Y,0,61,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,Y,298,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,Y,0,42,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,Y,0,35,272,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,Y,192,84,0,0,12,2008,WD,Normal,250000


In [9]:
HouseTrain_clean = HouseTrain[['Id', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'TotalBsmtSF', 'Heating', 'CentralAir', 'Electrical' ]]

In [10]:
HouseTrain.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'MiscVal',
       'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'SalePrice'],
      dtype='o

In [11]:
HouseTrain = HouseTrain.drop(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope','MiscVal'], axis=1)

In [12]:
HouseTrain.head()

,Id,LotArea,Street,Utilities,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,...,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,8450,Pave,AllPub,CollgCr,Norm,Norm,1Fam,2Story,7,...,548,Y,0,61,0,2,2008,WD,Normal,208500
1,2,9600,Pave,AllPub,Veenker,Feedr,Norm,1Fam,1Story,6,...,460,Y,298,0,0,5,2007,WD,Normal,181500
2,3,11250,Pave,AllPub,CollgCr,Norm,Norm,1Fam,2Story,7,...,608,Y,0,42,0,9,2008,WD,Normal,223500
3,4,9550,Pave,AllPub,Crawfor,Norm,Norm,1Fam,2Story,7,...,642,Y,0,35,272,2,2006,WD,Abnorml,140000
4,5,14260,Pave,AllPub,NoRidge,Norm,Norm,1Fam,2Story,8,...,836,Y,192,84,0,12,2008,WD,Normal,250000


In [13]:
HouseTest = pd.read_csv("D:/Shaniel/Data Engineer Projects/Data_Eng_Project_1/test.csv")

In [14]:
HouseTest.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [15]:
HouseTest_clean = HouseTest[['LotArea', 'BldgType', 'HouseStyle', 'Foundation', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'GarageType', 'GarageCars', 'SaleCondition']]

In [16]:
HouseTest_clean.head()

,LotArea,BldgType,HouseStyle,Foundation,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,GarageType,GarageCars,SaleCondition
0,11622,1Fam,1Story,CBlock,1,0,2,5,Attchd,1.0,Normal
1,14267,1Fam,1Story,CBlock,1,1,3,6,Attchd,1.0,Normal
2,13830,1Fam,2Story,PConc,2,1,3,6,Attchd,2.0,Normal
3,9978,1Fam,2Story,PConc,2,1,3,7,Attchd,2.0,Normal
4,5005,TwnhsE,1Story,PConc,2,0,2,5,Attchd,2.0,Normal


In [17]:
cur, conn = create_database() 

In [18]:
submission_table_create = ("""CREATE TABLE IF NOT EXISTS submission(
Id INT PRIMARY KEY, 
SalePrice INT)""")

In [19]:
cur.execute(submission_table_create)
conn.commit()

In [20]:
train_table_create = ("""CREATE TABLE IF NOT EXISTS train(
Id INT, 
LotArea INT, 
YearBuilt INT, 
YearRemodAdd INT, 
BsmtFinSF1 INT,
TotalBsmtSF INT,
Heating VARCHAR, 
CentralAir CHAR, 
Electrical VARCHAR 
)""")
cur.execute(train_table_create)
conn.commit()

In [21]:
cur.execute(train_table_create)
conn.commit()

In [22]:
test_table_create = ("""CREATE TABLE IF NOT EXISTS test(
LotArea INT, 
BldgType VARCHAR(10), 
HouseStyle VARCHAR(10),
Foundation VARCHAR(10),
FullBath INT,
HalfBath INT,
BedroomAbvGr INT,
TotRmsAbvGrd INT,
GarageType VARCHAR(10),
GarageCars VARCHAR,
SaleCondition CHAR(10))""")
cur.execute(test_table_create)
conn.commit()

In [23]:
submission_table_insert = ("""INSERT INTO submission(
Id, 
SalePrice)
VALUES (%s, %s)
""")

In [24]:
for i, row in HousePrices.iterrows():
    cur.execute(submission_table_insert, list(row))

In [25]:
conn.commit()

In [26]:
train_table_insert = ("""INSERT INTO train(
Id, 
LotArea, 
YearBuilt, 
YearRemodAdd, 
BsmtFinSF1,
TotalBsmtSF,
Heating, 
CentralAir, 
Electrical)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

In [27]:
for i, row in HouseTrain_clean.iterrows():
    cur.execute(train_table_insert, list(row))
    
conn.commit()

In [28]:
test_table_insert = ("""INSERT INTO test(
LotArea, 
BldgType, 
HouseStyle,
Foundation,
FullBath,
HalfBath,
BedroomAbvGr,
TotRmsAbvGrd,
GarageType,
GarageCars,
SaleCondition)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""")

In [29]:
for i, row in HouseTest_clean.iterrows():
    cur.execute(test_table_insert, list(row))
    
conn.commit()